# Hyperparameter Tuning of QRNN Models with AML SDK and HyperDrive

This notebook performs hyperparameter tuning of QRNN models with AML SDK and HyperDrive. It selects the best model by cross validation using the training data in the 6 forecast round. Specifically, it splits the training data into sub-training data and validation data. Then, it trains QRNN models with different sets of hyperparameters using the sub-training data and evaluate the pinball loss of each model with the validation data. The set of hyperparameters which yield the best cross validation pinball loss will be used to train models and forecast energy load across all 6 forecast rounds.

## Prerequisites
To run this notebook, you need to install AML SDK and its widget extension in your environment by running the following commands in a terminal. Before running the commands, you need to activate your environment by executing `activate <your env>` or `source activate <your env>` in a Linux VM.   
`pip3 install --upgrade azureml-sdk[notebooks,automl]`  
`jupyter nbextension install --py --user azureml.train.widgets`  
`jupyter nbextension enable --py --user azureml.train.widgets`  

To add the environment to your Jupyter kernels, you can do python3 -m ipykernel install --name <your env>. Besides, you need to create an Azure ML workspace and its configuration file (config.json) by following the 00.configuration.ipynb notebook.

In [32]:
import azureml
from azureml.core import Workspace, Run

# Check core SDK version number
print("Azure ML SDK Version: ", azureml.core.VERSION)

Azure ML SDK Version:  1.0.8


In [33]:
from azureml.telemetry import set_diagnostics_collection

# Opt-in diagnostics for better experience of future releases
set_diagnostics_collection(send_diagnostics=True)

Turning diagnostics collection on. 


## Initialize Workspace & Create an Azure ML Experiment

Initialize a [Workspace](https://docs.microsoft.com/azure/machine-learning/service/concept-azure-machine-learning-architecture#workspace) object from the existing workspace you created in the Prerequisites step. `Workspace.from_config()` below creates a workspace object from the details stored in `config.json`.

In [34]:
from azureml.core.workspace import Workspace

ws = Workspace.from_config()
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

Found the config file in: /data/home/tsperfadmin/Projects/zhouf/energy_forecast_fnn_model_v1/TSPerf/energy_load/GEFCom2017_D_Prob_MT_hourly/submissions/fnn/config.json
Workspace name: tsperfwszhouf
Azure region: eastus
Subscription id: 8dd18bd7-b9dc-4213-b367-cd6a7745bc88
Resource group: tsperf03


In [35]:
from azureml.core import Experiment

exp = Experiment(workspace=ws, name='tune_qrnn')

## Run Script on Batch AI Compute Target

### Create Batch AI cluster as compute target

In [36]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget

# choose a name for your cluster
compute_name =  "cpucompute"
compute_min_nodes = 0
compute_max_nodes = 16

vm_size = "STANDARD_D3_V2"


if compute_name in ws.compute_targets:
    compute_target = ws.compute_targets[compute_name]
    if compute_target and type(compute_target) is AmlCompute:
        print('found compute target. just use it. ' + compute_name)
else:
    print('creating a new compute target...')
    provisioning_config = AmlCompute.provisioning_configuration(vm_size = vm_size,
                                                                min_nodes = compute_min_nodes, 
                                                                max_nodes = compute_max_nodes)

    # create the cluster
    compute_target = ComputeTarget.create(ws, compute_name, provisioning_config)

    # can poll for a minimum number of nodes and for a specific timeout. 
    # if no min node count is provided it will use the scale settings for the cluster
    compute_target.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)

     # For a more detailed view of current AmlCompute status, use the 'status' property    
    print(compute_target.status.serialize())

found compute target. just use it. cpucompute


In [8]:
# If you have created the compute target, you should see one entry named 'gpucluster' of type BatchAI 
# in the workspace's compute_targets property.
#compute_targets = ws.compute_targets
#for name, ct in compute_targets.items():
#    print(name, ct.type, ct.provisioning_state)

### Configure Docker environment

In [9]:
from azureml.core.runconfig import EnvironmentDefinition
from azureml.core.conda_dependencies import CondaDependencies

env = EnvironmentDefinition()

env.python.user_managed_dependencies = False
env.python.conda_dependencies = CondaDependencies.create(conda_packages=['pandas', 'r-base', 'r-data.table', 'r-rjson', 'r-optparse', 'r-doparallel'],
                                                         pip_packages=['azure-cli-core<2.0.55'], 
                                                         python_version='3.6.2')
env.python.conda_dependencies.add_channel('conda-forge')
env.docker.enabled=True

### Upload data to default datastore

Upload the 6 round train data of Energy dataset to the workspace's default datastore, which will later be mounted on a AML Compute target for training. 

In [10]:
ds = ws.get_default_datastore()
print(ds.datastore_type, ds.account_name, ds.container_name)

AzureBlob tsperfwszhouf2444925548 azureml-blobstore-43291b4b-78d3-4c1a-94db-cdd97cf840c4


In [11]:
path_on_datastore = 'data'
ds.upload(src_dir='./data/features/train', target_path=path_on_datastore, overwrite=True, show_progress=True)

Uploading ./data/features/train/train_round_1.csv
Uploading ./data/features/train/train_round_2.csv
Uploading ./data/features/train/train_round_3.csv
Uploading ./data/features/train/train_round_4.csv
Uploading ./data/features/train/train_round_5.csv
Uploading ./data/features/train/train_round_6.csv
Uploaded ./data/features/train/train_round_2.csv, 1 files out of an estimated total of 6
Uploaded ./data/features/train/train_round_1.csv, 2 files out of an estimated total of 6
Uploaded ./data/features/train/train_round_3.csv, 3 files out of an estimated total of 6
Uploaded ./data/features/train/train_round_4.csv, 4 files out of an estimated total of 6
Uploaded ./data/features/train/train_round_5.csv, 5 files out of an estimated total of 6
Uploaded ./data/features/train/train_round_6.csv, 6 files out of an estimated total of 6


$AZUREML_DATAREFERENCE_ba6c8d9e0e2746b89ccdfe541ce65564

In [12]:
# Get data reference object for the data path
ds_data = ds.path(path_on_datastore)
print(ds_data)

$AZUREML_DATAREFERENCE_4a453b3cd99a48d2854d21795e4fd96b


### Create estimator

In [15]:
from azureml.core.runconfig import EnvironmentDefinition
from azureml.train.estimator import Estimator

script_folder = './'

script_params = {
    '--path': ds_data.as_mount(),
    '--cv_path': './',
    '--n_hidden_1': 5, 
    '--n_hidden_2': 5,
    '--iter_max': 3,
    '--penalty': 0
}

est = Estimator(source_directory=script_folder,
                script_params=script_params,
                compute_target=compute_target,
                use_docker=True,
                entry_script='aml_estimator.py',
                environment_definition=env)
# The above estimator defined using environment_definition has problem.

In [38]:
from azureml.train.estimator import Estimator

script_folder = './'

script_params = {
    '--path': ds_data.as_mount(),
    '--cv_path': './',
    '--n_hidden_1': 5, 
    '--n_hidden_2': 5,
    '--iter_max': 3,
    '--penalty': 0
}

est = Estimator(source_directory=script_folder,
                script_params=script_params,
                compute_target=compute_target,
                use_docker=True,
                entry_script='aml_estimator.py',
                conda_packages=['pandas', 'r-base', 'r-data.table', 'r-rjson', 'r-doparallel'],
                pip_packages=['azure-cli-core<2.0.55'])

### Submit job

In [39]:
# Submit job to Batch AI cluster
run_batchai = exp.submit(config=est)

### Check job status

In [43]:
run_batchai.get_details()

{'runId': 'tune_qrnn_1547986964549',
 'target': 'cpucompute',
 'status': 'Running',
 'startTimeUtc': '2019-01-20T12:30:57.444364Z',
 'properties': {'azureml.runsource': 'experiment',
  'ContentSnapshotId': '857cee45-9475-4124-87d7-2483f6c7716f'},
 'runDefinition': {'Script': 'aml_estimator.py',
  'Arguments': ['--path',
   '$AZUREML_DATAREFERENCE_dced8724e0954ff78a554a6145adca80',
   '--cv_path',
   './',
   '--n_hidden_1',
   '5',
   '--n_hidden_2',
   '5',
   '--iter_max',
   '3',
   '--penalty',
   '0'],
  'SourceDirectoryDataStore': None,
  'Framework': 0,
  'Communicator': 0,
  'Target': 'cpucompute',
  'DataReferences': {'dced8724e0954ff78a554a6145adca80': {'DataStoreName': 'workspaceblobstore',
    'Mode': 'Mount',
    'PathOnDataStore': 'data',
    'PathOnCompute': None,
    'Overwrite': False}},
  'JobName': None,
  'AutoPrepareEnvironment': True,
  'MaxRunDurationSeconds': None,
  'NodeCount': 1,
  'Environment': {'Python': {'InterpreterPath': 'python',
    'UserManagedDepend

In [44]:
run_batchai.get_metrics()

{}

## Tune Hyperparameters using HyperDrive

In [24]:
from azureml.core.runconfig import EnvironmentDefinition
from azureml.train.estimator import Estimator
from azureml.train.hyperdrive import *

script_folder = './'

script_params = {
    '--path': ds_data.as_mount(),
    '--cv_path': './',
    '--n_hidden_1': 5, 
    '--n_hidden_2': 5,
    '--iter_max': 3,
    '--penalty': 0
}


est = Estimator(source_directory=script_folder,
                script_params=script_params,
                compute_target=compute_target,
                use_docker=True,
                entry_script='aml_estimator.py',
                environment_definition=env)

ps = GridParameterSampling({
    '--n_hidden_1': choice(4, 8), 
    '--n_hidden_2': choice(4, 8),
    '--iter_max': choice(1, 2, 4, 6, 8, 10),
    '--penalty': choice(0, 0.001),
})

htc = HyperDriveRunConfig(estimator=est, 
                          hyperparameter_sampling=ps, 
                          primary_metric_name='average pinball loss', 
                          primary_metric_goal=PrimaryMetricGoal.MINIMIZE, 
                          max_total_runs=48,
                          max_concurrent_runs=16)
# The above estimator has problem

In [47]:
from azureml.train.estimator import Estimator
from azureml.train.hyperdrive import *

script_folder = './'

script_params = {
    '--path': ds_data.as_mount(),
    '--cv_path': './',
    '--n_hidden_1': 5, 
    '--n_hidden_2': 5,
    '--iter_max': 3,
    '--penalty': 0
}


est = Estimator(source_directory=script_folder,
                script_params=script_params,
                compute_target=compute_target,
                use_docker=True,
                entry_script='aml_estimator.py',
                conda_packages=['pandas', 'r-base', 'r-data.table', 'r-rjson', 'r-doparallel'],
                pip_packages=['azure-cli-core<2.0.55'])

ps = GridParameterSampling({
    '--n_hidden_1': choice(4, 8, 16), 
    '--n_hidden_2': choice(4, 8),
    '--iter_max': choice(1, 2, 4, 6, 8, 10),
    '--penalty': choice(0, 0.001),
})

htc = HyperDriveRunConfig(estimator=est, 
                          hyperparameter_sampling=ps, 
                          primary_metric_name='average pinball loss', 
                          primary_metric_goal=PrimaryMetricGoal.MINIMIZE, 
                          max_total_runs=72,
                          max_concurrent_runs=16)

In [49]:
htr = exp.submit(config=htc)

OSError: [Errno 28] No space left on device: '/tmp/tmpr6u9wd_7'

In [ ]:
RunDetails(htr).show()

In [ ]:
htr.get_details()

In [ ]:
htr.get_metrics()

In [ ]:
best_run = htr.get_best_run_by_primary_metric()
parameter_values = best_run.get_details()['runDefinition']['Arguments']
print(parameter_values)